In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [4]:
# Step 1 load data
dataframe = pd.read_csv("data.csv") 
# remove the features we don't care about
dataframe = dataframe.drop(["index", "price", "sq_price"], axis=1)
dataframe = dataframe[0:10]
dataframe

,area,bathrooms
0,2104.0,3.0
1,1600.0,3.0
2,2400.0,3.0
3,1416.0,2.0
4,3000.0,4.0
5,1985.0,4.0
6,1534.0,3.0
7,1427.0,3.0
8,1380.0,3.0
9,1494.0,3.0


In [8]:
# Step 2 - add labels
# 1 is good buy 0 is bad buy
dataframe.loc[:, ("y1")] = [1,1,1,0,0,1,0,1,1,1]
# y2 is negation of y1, opposite
dataframe.loc[:, ("y2")] = dataframe["y1"] == 0
# turn TRUE/FALSE values to 1s and 0s
dataframe.loc[:, ("y2")] = dataframe["y2"].astype(int)
dataframe

,area,bathrooms,y1,y2
0,2104.0,3.0,1,0
1,1600.0,3.0,1,0
2,2400.0,3.0,1,0
3,1416.0,2.0,0,1
4,3000.0,4.0,0,1
5,1985.0,4.0,1,0
6,1534.0,3.0,0,1
7,1427.0,3.0,1,0
8,1380.0,3.0,1,0
9,1494.0,3.0,1,0


In [12]:
# Step 3 - prepare data for tensorflow (tensors)
# tensors are generic versions of vectors and matrices
# vector - is a list of numbers
# matrix - is a list of lists of numbers
# list of list of list of numbers (3D tensor)
# ... (and so on)

# convert features to input tensor
inputX = dataframe.loc[:, ["area", "bathrooms"]].as_matrix()
# convert labels to input tensors
inputY = dataframe.loc[:, ["y1", "y2"]].as_matrix()
(inputX, inputY)

(array([[  2.10400000e+03,   3.00000000e+00],
        [  1.60000000e+03,   3.00000000e+00],
        [  2.40000000e+03,   3.00000000e+00],
        [  1.41600000e+03,   2.00000000e+00],
        [  3.00000000e+03,   4.00000000e+00],
        [  1.98500000e+03,   4.00000000e+00],
        [  1.53400000e+03,   3.00000000e+00],
        [  1.42700000e+03,   3.00000000e+00],
        [  1.38000000e+03,   3.00000000e+00],
        [  1.49400000e+03,   3.00000000e+00]]), array([[1, 0],
        [1, 0],
        [1, 0],
        [0, 1],
        [0, 1],
        [1, 0],
        [0, 1],
        [1, 0],
        [1, 0],
        [1, 0]]))

In [13]:
# Step 4 - write out our hyperparameters
learning_rate = 0.000001
training_epochs = 2000
display_steps = 50
n_samples = inputY.size


In [20]:
# Step 5 - Create our computation graph/neural network
# for our feature input tensor, None means any number of examples
# placeholders are placeholders for data in our computation graph
x = tf.placeholder(tf.float32, [None, 2])

# Create weights
# 2x2 float matrix, that we'll keep updating through the training process
# variables in tensorflow hold and update parameters
# in memory buffers containing tensors
W = tf.Variable(tf.zeros([2,2]))

# add biases (example is b in y = mx + b, b is the bias)
b = tf.Variable(tf.zeros([2]))

# multiply our weights by our inputs
# weights are how we govern how data flows in our computation graph
# multiply input by weights and add biases
y_values = tf.add(tf.matmul(x, W), b)

# softmax is another way to say sigmoid
# apply softmax to value we just created
# softmax is our activation function
y = tf.nn.softmax(y_values)

# feed in a matrix of labels
y_ = tf.placeholder(tf.float32, [None, 2])

In [21]:
# Step 6 perform training
# create our cost function, mean square error
# reduce sum computes the sum of elements across dimensions of a tensor
cost = tf.reduce_sum(tf.pow(y_ - y, 2)) / (2 * n_samples)

optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [22]:
# initialize variables and tensorflow sesssion
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [ ]:
# training loop
for i in range(training_epochs):
    sess.run(optimizer, feed_dict)={x:inputX, y_:inputY}
    
    # write out logs of training
    if (i) % display_step == 0:
        cc = sess.run(cost, feed_dict{x: inputX, y_:inputY})
        print("training steps:")